# Huntsman California Housing Price Prediction
**Author:** Kate Huntsman
**Date:** March 10th, 2025  
**Objective:** Predict the median house price in California using available housing features.


## Introduction
This project leverages the California housing dataset to forecast property prices, utilizing factors like median income, average room count, and the age of the houses. My approach includes data preprocessing, training a linear regression model, and experimenting with techniques to enhance its accuracy.

## Imports
In the code cell below, import the necessary Python libraries for this notebook.  

In [ ]:
# This is a Python cell
# All imports should be at the top of the notebook
# This cell will be executed when the notebook is loaded

import sys
print(sys.executable)

# Import pandas for data manipulation and analysis
import pandas as pd

# Import numpy for numerical operations
import numpy as np

# Import matplotlib for creating static visualizations
import matplotlib.pyplot as plt

# Import seaborn for statistical data visualization (built on matplotlib)
import seaborn as sns

# Import the California housing dataset from sklearn
from sklearn.datasets import fetch_california_housing

# Import train_test_split for splitting data into training and test sets
from sklearn.model_selection import train_test_split

# Import LinearRegression for building a linear regression model
from sklearn.linear_model import LinearRegression

# Import performance metrics for model evaluation
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

## Section 1. Load and Explore the Data

### 1.1 Load the dataset and display the first 10 rows
Load the California housing dataset directly from `scikit-learn`.
- The `fetch_california_housing` function returns a dictionary-like object with the data.
- Convert it into a pandas DataFrame.
- Display just the first 10 rows using `head()`.

Example code:

data = fetch_california_housing(as_frame=True)
data_frame = data.frame

data_frame.head(10)

In [ ]:
# Load the data
data = fetch_california_housing(as_frame=True)
data_frame = data.frame

# Might be large. Display just the first 10 rows (you can change this number)
data_frame.head(10)

### 1.2 Check for missing values and display summary statistics

In the cell below:
1. Use `info()` to check data types and missing values.
2. Use `describe()` to see summary statistics.
3. Use `isnull().sum()` to identify missing values in each column.

Example code:

data_frame.info()

data_frame.describe()

data_frame.isnull().sum()

In [ ]:
# Fetch the California housing dataset
california_housing = fetch_california_housing(as_frame=True)
data_frame = california_housing.frame

# 1. Use info() to check data types and missing values
print("Data Types and Missing Values:")
data_frame.info()

# 2. Use describe() to see summary statistics
print("\nSummary Statistics:")
print(data_frame.describe())

# 3. Use isnull().sum() to identify missing values in each column
print("\nMissing Values in Each Column:")
print(data_frame.isnull().sum())

Analysis: 

1) How many data instances (also called data records or data rows) are there? 20,640

2) How many features (also columns or attributes) are there? 9

3) What are the names of the features? ("Feature" is used most often in ML projects.) 
MedInc
HouseAge
AveRooms
AveBedrms
Population
AveOccup
Latitude
Longitude
MedHouseVal

4) Which features are numeric? all 9 are numeric (float64 data type)

5) Which features are categorical (non-numeric)? none

6) Are there any missing values? How should they be handled? Should we delete a sparsely populated column? Delete an incomplete data row? Substitute with a different value? 
All columns have 20640 non-null entries, meaning there are no missing values in the dataset.
Thus, no handling of missing values is needed (no need to delete rows or columns, or substitute values).

7) What else do you notice about the dataset? Are there any data issues? 
The dataset is clean with no missing values, making it straightforward to work with. All features are numeric, and there are no categorical attributes. Some features, like `Population`, have a wide range of values, which might indicate potential outliers or skewness. It’s worth investigating possible correlations between features to avoid multicollinearity in future analysis. Overall, the dataset appears ready for analysis, though further exploration for outliers or data transformations may be helpful.

## Section 2. Visualize Feature Distributions
### 2.1 Create histograms, boxplots, and scatterplots

- Create histograms for all numeric features using `data_frame.hist()` with 30 bins.
- Create a boxenplots using `sns.boxenplot()`.
- Create scatter plots using `sns.pairplot()`.

First, histograms

Generate histograms for all numerical columns

Example code:

data_frame.hist(bins=30, figsize=(12, 8))

plt.show()

In [ ]:
# Create histograms for all numeric features
data_frame.hist(bins=30, figsize=(12, 8))

# Show the plot
plt.show()



Generate one Boxenplot for each column (good for large datasets)

Example code:

for column in data_frame.columns:
    plt.figure(figsize=(6, 4))
    sns.boxenplot(data=data_frame[column])
    plt.title(f'Boxenplot for {column}')
    plt.show()

In [ ]:
# Create boxenplots for each numeric feature
plt.figure(figsize=(12, 8))
for i, column in enumerate(data_frame.select_dtypes(include=[np.number]).columns):
    plt.subplot(3, 3, i + 1)
    sns.boxenplot(data=data_frame, x=column)
    plt.title(f"Boxenplot for {column}")
plt.tight_layout()
plt.show()

Third - Scatter Plots

Generate all Scatter plots (there is a LOT of data, so this will take a while)

Comment out after analysis to speed up the notebook.

Example code:

sns.pairplot(data_frame)

plt.show()

In [ ]:
# Create pairplot for scatter plots between numeric features
sns.pairplot(data_frame)
plt.show()


## Section 3. Feature Selection and Justification
### 3.1 Choose two input features for predicting the target

- Select `MedInc` and `AveRooms` as predictors.
- Select `MedHouseVal` as the target variable.

In the following, 
X is capitalized because it represents a matrix (consistent with mathematical notation).
y is lowercase because it represents a vector (consistent with mathematical notation).


First:
- Create a list of contributing features and the target variable
- Define the target feature string (the variable we want to predict)
- Define the input DataFrame
- Define the output DataFrame


Example code:

features: list = ['MedInc', 'AveRooms']

target: str = 'MedHouseVal'

df_X = data_frame[features]

df_y = data_frame[target]


In [ ]:
# List of contributing features (input variables)
features = ['MedInc', 'AveRooms']

# Define the target variable (the variable we want to predict)
target = 'MedHouseVal'

# Define the input DataFrame (X) using the selected features
df_X = data_frame[features]

# Define the output DataFrame (y) using the target variable
df_y = data_frame[target]

## Section 4. Train a Linear Regression Model
### 4.1 Split the data
Split the dataset into training and test sets (80% train / 20% test).

Call train_test_split() by passing in: 

- df_X – Feature matrix (input data) as a pandas DataFrame
- y – Target values as a pandas Series
- test_size – Fraction of data to use for testing (e.g., 0.1 = 10%)
- random_state – Seed value for reproducible splits

We'll get back four return values:

- X_train – Training set features (DataFrame)
- X_test – Test set features (DataFrame)
- y_train – Training set target values (Series)
- y_test – Test set target values (Series)


Example code:

X_train, X_test, y_train, y_test = train_test_split(
    df_X, df_y, test_size=0.2, random_state=42)


In [ ]:
# Split the data into training and test sets (80% train, 20% test)
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    df_X, df_y, test_size=0.2, random_state=42)


### 4.2 Train the model
Create and fit a `LinearRegression` model.

LinearRegression – A class from sklearn.linear_model that creates a linear regression model.

model – An instance of the LinearRegression model. This object will store the learned coefficients and intercept after training.

fit() – Trains the model by finding the best-fit line for the training data using the Ordinary Least Squares (OLS) method.

X_train – The input features used to train the model.

y_train – The target values used to train the model.


Example code:


model = LinearRegression()

model.fit(X_train, y_train)


In [ ]:
# Import LinearRegression from sklearn
from sklearn.linear_model import LinearRegression

# Create an instance of the LinearRegression model
model = LinearRegression()

# Train the model using the training data
model.fit(X_train, y_train)

Make predictions for the test set.

The model.predict() method applies this equation to the X test data to compute predicted values.

y_pred = model.predict(X_test)

y_pred contains all the predicted values for all the rows in X_test based on the linear regression model.


Example code:

y_pred = model.predict(X_test)


In [ ]:
# Make predictions on the test set
y_pred = model.predict(X_test)

### 4.3 Report R^2, MAE, RMSE
Evaluate the model using R^2, MAE, and RMSE.

First:

- Coefficient of Determination (R^2) - This tells you how well the model explains the variation in the target variable. A value close to 1 means the model fits the data well; a value close to 0 means the model doesn’t explain the variation well.


Example code:
  
r2 = r2_score(y_test, y_pred)

print(f'R²: {r2:.2f}')



In [ ]:
# Import r2_score from sklearn.metrics
from sklearn.metrics import r2_score

# Calculate R² (Coefficient of Determination)
r2 = r2_score(y_test, y_pred)

# Print the R² value
print(f'R²: {r2:.2f}')


Second:

- Mean Absolute Error (MAE) - This is the average of the absolute differences between the predicted values and the actual values. A smaller value means the model’s predictions are closer to the actual values.


Example code:

mae = mean_absolute_error(y_test, y_pred)

print(f'MAE: {mae:.2f}')




In [ ]:
# Import mean_absolute_error from sklearn.metrics
from sklearn.metrics import mean_absolute_error

# Calculate Mean Absolute Error (MAE)
mae = mean_absolute_error(y_test, y_pred)

# Print the MAE value
print(f'MAE: {mae:.2f}')


Third:

- Root Mean Squared Error (RMSE) - This is the square root of the average of the squared differences between the predicted values and the actual values. It gives a sense of how far the predictions are from the actual values, with larger errors having more impact.

Example code:

rmse = root_mean_squared_error(y_test, y_pred)

print(f'RMSE: {rmse:.2f}')


In [ ]:
# Import mean_squared_error from sklearn.metrics to calculate RMSE
from sklearn.metrics import mean_squared_error
import numpy as np

# Calculate Root Mean Squared Error (RMSE)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

# Print the RMSE value
print(f'RMSE: {rmse:.2f}')